In [1]:
import PyPDF2
import os
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
import en_core_web_sm
nlp = en_core_web_sm.load()
import re
import string
import pandas as pd
from tika import parser 
from io import StringIO
from bs4 import BeautifulSoup
from spacy.matcher import PhraseMatcher

# Converting PDFs to text and organising by applicant 

In [2]:
#get_text is a function that takes in a directory and returns a list of each PDF's data. 
#Each PDF's data is represented by a list that has separated the PDF by page.
def get_text(directory_name): 
    files = []
    for file in os.listdir(directory_name):
        if file.endswith(".pdf"): 
            file_with_path = directory_name + os.sep + file
            raw = parser.from_file(file_with_path, xmlContent=True)
            content = raw['content']

            file_data = []
            
            xhtml_data = BeautifulSoup(content)
            for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
                
                _buffer = StringIO()
                _buffer.write(str(content))
                parsed_content = parser.from_buffer(_buffer.getvalue())
                if parsed_content['content'] != None:
                    #data cleaning of tabs, spaces and dashes
                    parsed_content['content'] = parsed_content['content'].replace("\xa0", " ")
                    parsed_content['content'] = parsed_content['content'].casefold()
                    parsed_content['content'] = parsed_content['content'].replace('\xad', '')
                    parsed_content['content'] = re.sub(r'\t', '', parsed_content['content'])

                    file_data.append({'id': 'page_'+str(page+1), 'content': parsed_content['content']})
        files.append(file_data)
    return files

def get_applicants(pdf_data):
    p1_text = pdf_data[0]['content']    
    index_start = p1_text.find("included in this document:")   
    index_end = p1_text.find("job details:")
    names_with_line_breaks = p1_text[index_start+27:index_end-3] 
    return names_with_line_breaks.splitlines(0)

def organise_by_applicant(lst_of_pdfs): #intended input is the lst_of_pdfs from get_text
    output = []
    for pdf in lst_of_pdfs: 
        list_of_applicants = get_applicants(pdf)  
        applicant_index = 0
        application_form = ""
        applicant_cv = ""
        curr_pdf = []
        flag = False
        cv_flag = False
        for pg in pdf:
            #data cleaning of line breaks
            pg['content'] = pg['content'].replace("\n\n\n", " ")
            pg['content'] = pg['content'].replace("\n", " ")
            if len(list_of_applicants) == 1 and flag == False:
                flag = True
                continue
            elif "applicant address:" in pg['content']: #reached 1st page of an applicant
                if flag and len(list_of_applicants) > 1: #next applicant reached
                    curr_pdf.append({'name': list_of_applicants[applicant_index], 'application_form': application_form, 'cv': applicant_cv})
                    applicant_index += 1
                    application_form = ""
                    applicant_cv = ""
                    cv_flag = False
                else:
                    application_form = application_form+ pg['content']
                    flag = True
            else:
                if flag:
                    if pg['content'].find("clauses above and that i agree to all of them") != -1:
                        cv_flag = True
                    if cv_flag and pg['content'].find("clauses above and that i agree to all of them") == -1:
                        applicant_cv = applicant_cv + pg['content']
                    else:
                        application_form = application_form + pg['content']
        if len(list_of_applicants) == 1:
            curr_pdf.append({'name': list_of_applicants[0], 'application_form': application_form, 'cv': applicant_cv})
        output.append(curr_pdf)        
    return output       
    

# input required: directory, fresh grad year 

In [3]:
#Give a directory with your PDFs of CVs. Can handle Bulk combined CVs as well as single CVs. 
directory = "C:/Users/caitl/Documents/1uni/intern/accounts payable"
fresh_grad_year = 2020

In [4]:
dir_pdfs = get_text(directory)
organised_pdfs_list = organise_by_applicant(dir_pdfs)

In [5]:
def organise_education(lst_pdfs_by_applicant):
    for pdf in lst_pdfs_by_applicant:
        for applicant in pdf:
            education_count = 1
            edu_applicant = []
            while (applicant['application_form'].find("education " + str(education_count)) != -1):
                edu_details = {}
                edu_details['cert_type'] = ""
                edu_details['discipline'] = ""
                edu_details['school'] = ""
                edu_details['year_finished'] = 0
                index_start = applicant['application_form'].find("education " + str(education_count))
                
                check_tertiary_start = applicant['application_form'].find("type of education", index_start) + 18
                if applicant['application_form'][check_tertiary_start:check_tertiary_start+8] == "tertiary":
                    check_completed_start = applicant['application_form'].find("education status", index_start) + 17
                    if applicant['application_form'][check_completed_start:check_completed_start+9] != "completed": #ignore incomplete qualifications
                        education_count += 1
                        continue
                    
                    cert_type_start = applicant['application_form'].find("expected certificate type", index_start) + 26
                    cert_type = applicant['application_form'][cert_type_start:applicant['application_form'].find(" discipline", index_start)]
                    edu_details['cert_type'] = cert_type.strip() 
                    discipline = applicant['application_form'][applicant['application_form'].find("discipline", index_start) + 11:applicant['application_form'].find(" second discipline", index_start)]
                    edu_details['discipline'] = discipline.strip()
                    school = applicant['application_form'][applicant['application_form'].find("institution", index_start) + 12:applicant['application_form'].find(" education status", index_start)]
                    edu_details['school'] = school.strip()
                    year_finished = applicant['application_form'][applicant['application_form'].find("year finished (actual or anticipated)", index_start) + 38:applicant['application_form'].find("year finished (actual or anticipated)", index_start) + 42]
                    edu_details['year_finished'] = int(year_finished.strip())
                    
                else: #ignore non-tertiary qualifications
                    education_count += 1
                    continue
                
                education_count += 1
                edu_applicant.append(edu_details)
            
            applicant['education'] = edu_applicant
    return lst_pdfs_by_applicant
            
            
organised_education = organise_education(organised_pdfs_list)

In [6]:
diploma_only = pd.read_csv("diploma.csv")
any_tertiary = pd.read_csv("tertiary_all.csv")
bach_and_above = pd.read_csv("bach_and_above.csv")

def filter_edu_level(filter_choice, edu_pdfs): 
    if filter_choice != 0 and filter_choice != 1 and filter_choice != 2 and filter_choice != 3 and filter_choice != 4:
        print("PLEASE SELECT A VALID CHOICE: \n0 = any tertiary (dip & above); \n1 = bach & above; \n2 = diploma (local poly) only; \n3 = fresh grads; \n4: diploma or fresh grads")
    else:
        #filter_choice options: 0 = any tertiary (dip & above); 1 = bach & above; 2 = diploma (local poly) only; 3 = fresh grads; 4 = dip or fresh grads 
        output = []
        for pdf in edu_pdfs:
            filtered = []
            for applicant in pdf:
                flag_0 = False
                flag_1 = False
                flag_2 = False
                flag_3 = False
                for edu in applicant['education']:
                    if edu['cert_type'] in any_tertiary['cert_type'].values:
                        flag_0 = True
                    if edu['cert_type'] in diploma_only['cert_type'].values:
                        flag_2 = True
                    if edu['cert_type'] in bach_and_above['cert_type'].values: 
                        flag_1 = True
                        if edu['year_finished'] == 2020:
                            flag_3 = True
                    
                #flagging done
                if filter_choice == 0: #any tertiary
                    if flag_0 + flag_1 + flag_2 > 0:
                        filtered.append(applicant) 
                elif filter_choice == 1:
                    if flag_1:
                        filtered.append(applicant) 
                elif filter_choice == 2:
                    if flag_2 and flag_1 == False:
                        filtered.append(applicant) 
                elif filter_choice == 3:
                    if flag_3:
                        filtered.append(applicant) 
                elif filter_choice == 4:
                    if flag_1:
                        if flag_3:
                            filtered.append(applicant)
                    else: 
                        if flag_2:
                            filtered.append(applicant)
            if len(filtered) > 0:
                output.append(filtered) 
        return output
    

In [7]:
filtered_edu_level = filter_edu_level(4, organised_education)


# change csv of acceptable majors where needed

In [8]:
all_majors = pd.read_csv("all_majors.csv")
biz = pd.read_csv("majors_biz.csv")
biz
def filter_course(filtered_edu_level_pdfs, majors_df, names): #names is a boolean True if only want names, False if want cvs
    output = []
    for pdf in filtered_edu_level_pdfs:
        filtered = []
        for applicant in pdf:
            flag = False
            for edu in applicant['education']:
                if edu['discipline'] in majors_df['major'].values:
                    flag = True
            if flag:
                filtered.append(applicant) 
                if names:
                    output.append(applicant['name'])
        if len(filtered) > 0 and names == False:
            output.append(filtered) 
    if names:
        for name in output:
            print(name.title())
    else:
        return output

filtered_edu = filter_course(filtered_edu_level, biz, False)

# filter by schools

In [9]:
local_schools = pd.read_csv("schools.csv")
def filter_school(filtered_edu_level_pdfs):
    #filters for local polys & unis
    output = []
    for pdf in filtered_edu_level_pdfs:
        filtered = []
        for applicant in pdf:
            flag = False
            for edu in applicant['education']:
                if edu['school'] in local_schools['school'].values:
                    flag = True
            if flag:
                filtered.append(applicant) 
        if len(filtered) > 0:
            output.append(filtered) 
    
    return output

filtered_local_schools = filter_school(filtered_edu_level)

In [10]:
def organise_work(lst_pdfs_by_applicant):
    for pdf in lst_pdfs_by_applicant:
        for applicant in pdf:
            work_count = 1
            work_exp = [] 
            work_years = 0
            work_months = 0
            while (applicant['application_form'].find("employer " + str(work_count)) != -1 or work_count == 1):
                work_details = {}
                if work_count == 1:
                    index_start = applicant['application_form'].find("current/most recent employer")
                else:
                    index_start = applicant['application_form'].find("employer " + str(work_count))
                index_end = applicant['application_form'].find("brief description of job duties", index_start)
                
                years_start = applicant['application_form'].find("time in role", index_start, index_end) + 12
                years_end = applicant['application_form'].find("years ", years_start, index_end) 
                if years_end == -1:
                    years_end = applicant['application_form'].find("year ", years_start, index_end) 
                    if years_end == -1:
                        years = 0
                        years_end = years_start + 1
                    else: 
                        years = int(applicant['application_form'][years_start:years_end].strip())
                        years_end += 5
                else:
                    years = int(applicant['application_form'][years_start:years_end].strip())
                    years_end += 6                    
                months_end = applicant['application_form'].find("month", years_end) 
                months = int(applicant['application_form'][years_end:months_end].strip())
                work_count += 1
                work_details['years'] = years
                work_details['months'] = months
                job_title_start = applicant['application_form'].find("job title", index_start) + 9
                job_title_end = applicant['application_form'].find("company name", job_title_start)
                work_details['job title'] = applicant['application_form'][job_title_start:job_title_end].strip()
                job_desc_start = applicant['application_form'].find("brief description of job duties & achievements (to include type of employment, eg. permanent/contract/casual)", index_start) + 110 
                job_desc_end = applicant['application_form'].find("company industry", job_desc_start)
                work_details['job_description'] = applicant['application_form'][job_desc_start:job_desc_end].strip()
                work_years += years
                work_months += months
                work_exp.append(work_details)
            applicant['work'] = work_exp
            applicant['total_work_exp'] = work_years + int(work_months/12)
    return lst_pdfs_by_applicant
            
            
filtered_local_schools = organise_work(filtered_local_schools)

# filter minimum working exp

In [22]:
def filter_working_exp(lst_pdfs_by_applicant, min_years, names_bool):
    output = []
    for pdf in lst_pdfs_by_applicant:
        filtered = []
        for applicant in pdf:
        
            if applicant['total_work_exp'] >= min_years:
                filtered.append(applicant)
                if names_bool:
                    output.append(applicant['name'])
        if len(filtered) > 0 and names_bool == False:
            output.append(filtered) 
    if names_bool:
        count = 0
        for name in output:
            newStr = name.title()
            for ele in newStr.split():
                for i in range(-1,len(ele),2):
                    n=ele[i]
                    newStr=newStr.replace(n, "*")
            output[count] = newStr
            count+=1
    return output

In [23]:
for name in filter_working_exp(filtered_local_schools,2,True):
    print(name)

N*r*i**** B** A*m**
Y* S*e** L**
N** S******* B*n** M***m** A*******
E*e*** T**
J*s**** T** H** L**
